In [2]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from aws.dynamicframe import DynamicFrame

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


In [6]:
# retorna o nome das tabelas que foram atualizadas pela última vez
def return_last_updated_tables(response):
    tables = sorted(response['TableList'], key=lambda k: k['UpdateTime'], reverse=True)
    last_updated_tables = []
    for i in range(len(tables)):
        table = tables[i]
        last_updated_tables.append(table['Name'])
    return last_updated_tables

# salva as tabelas no bucket cleaned
def save_in_cleaned(last_updated_tables):
    for table in last_updated_tables:
        table_name=table
        dynamic_frame = glueContext.create_dynamic_frame.from_catalog(
            database = 'olist-project-raw',
            table_name = table_name,
            transformation_ctx = 'read_csv'
        )
        parquet_frame = dynamic_frame.toDF().write.format('parquet').save('s3://olist-project-dw/cleaned/' + table_name + '.parquet')
        return print('Tabelas carregadas com sucesso!')

['product_category_name_translation_csv', 'olist_sellers_dataset_csv', 'olist_orders_dataset_csv', 'olist_products_dataset_csv', 'olist_order_payments_dataset_csv', 'olist_order_reviews_dataset_csv', 'olist_order_items_dataset_csv', 'olist_customers_dataset_csv', 'olist_geolocation_dataset_csv']


In [ ]:
try:
    glue = boto3.client('glue')
    response = glue.get_tables(DatabaseName='olist-project-raw')
except Exception as e:
    print("Ocorreu um erro ao acessar o AWS Glue:")
    print(str(e))

In [ ]:
try:
    last_updated_tables = return_last_updated_tables(response)
    save_in_cleaned(last_updated_tables)
except Exception as e:
    print("Uma excessão ocorreu ao salvar as tabelas no bucket 'cleaned': ")
    print(str(e))